100%|██████████| 2231142/2231142 [00:52<00:00, 42467.49it/s]


## Trening

In [1]:
#import prodigy
import spacy
import csv
import os

from pathlib import Path
#from prodigy.models.ner import EntityRecognizer

nlp = spacy.load("en_core_web_sm")
#name_dict, desc_dict = load_food_entities()
#kb = spacy.kb.KnowledgeBase(vocab=nlp.vocab, entity_vector_length=300)

import re
import itertools
from itertools import chain, combinations
import warnings
warnings. filterwarnings('ignore')
stopwords = nlp.Defaults.stop_words
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
ps = PorterStemmer()
def normalize_name(name):
    name = re.sub(r'[^a-zA-Z]', ' ', name)
    name = re.sub(r'\s+', ' ', name)
    name = name.lower()
    name = ' '.join([token for token in name.split(' ') if token not in stopwords])
    stemmed_name = " ".join([ps.stem(token.text) for token in nlp(name)])
    return stemmed_name

In [2]:
import pandas as pd
df = pd.read_csv("recipeNLG_with_entities.csv")

In [3]:
import json
from tqdm import tqdm

corpus = []
json_fail_count = 0
json_fail = []
for ingredients_entities in tqdm(df["ingredients_entities"].tolist()):
    sentence = []
    try:
        for entity in json.loads(ingredients_entities):
            if entity["type"] == "FOOD":
                sentence.append("_".join(normalize_name(entity["entity"]).split(" ")))
        corpus.append(" ".join(sentence).lower())
    except:
        json_fail.append(ingredients_entities)
        json_fail_count += 1

100%|██████████| 2231142/2231142 [22:45:56<00:00, 27.22it/s]   


In [4]:
with open('dataset_new_forfasttext_lower.csv', 'w') as csvfile:
    for i in corpus:
        csvfile.write(i.lower())
        csvfile.write("\n")

In [25]:
import fasttext
model = fasttext.train_unsupervised('dataset_new_forfasttext_instructions.csv',thread=8, minn=4, maxn=6, dim=50, verbose=0)

In [26]:
model.save_model("dataset_new_instructions.bin")

In [7]:
len(model.words)

32781

## Wczytanie modelu

In [33]:
import fasttext
model = fasttext.load_model("dataset_new.bin")
model.get_nearest_neighbors('chicken')

[(0.9069104194641113, 'meat_chicken'),
 (0.905862033367157, 'chicken_chicken'),
 (0.9000569581985474, 'citi_chicken'),
 (0.8813421130180359, 'mix_chicken'),
 (0.8800981640815735, 'mix_chicken_piec'),
 (0.8751397132873535, 'chicken_cut'),
 (0.8745503425598145, 'chicken_cook'),
 (0.8701257705688477, 'chicken_hen'),
 (0.8680923581123352, 'chicken_leftov'),
 (0.8677511215209961, 'mixin_chicken')]

In [30]:
model.words

['</s>',
 'sugar',
 'salt',
 'pepper',
 'egg',
 'onion',
 'butter',
 'water',
 'milk',
 'garlic',
 'flour',
 'oliv_oil',
 'vanilla',
 'cinnamon',
 'margarin',
 'chicken',
 'tomato',
 'purpos_flour',
 'bake_powder',
 'lemon_juic',
 'cream_chees',
 'sour_cream',
 'bake_soda',
 'carrot',
 'potato',
 'parsley',
 'celeri',
 'cheddar_chees',
 'veget_oil',
 'vanilla_extract',
 'beef',
 'pecan',
 'mayonnais',
 'oil',
 'parmesan_chees',
 'bell_pepper',
 'ginger',
 'nutmeg',
 'soy_sauc',
 'vinegar',
 'lemon',
 'mushroom',
 'chicken_broth',
 'kosher_salt',
 'honey',
 'nut',
 'rice',
 'garlic_powder',
 'mustard',
 'bacon',
 'basil',
 'oregano',
 'worcestershir_sauc',
 'pineappl',
 'paprika',
 'cornstarch',
 'walnut',
 'wine',
 'chocol_chip',
 'thyme',
 'shorten',
 'cumin',
 'raisin',
 'coconut',
 'banana',
 'buttermilk',
 'chees',
 'chocol',
 'almond',
 'appl',
 'chili_powder',
 'bean',
 'heavi_cream',
 'cilantro',
 'orang_juic',
 'peanut_butter',
 'cool_whip',
 'strawberri',
 'soda',
 'corn',
 'c

In [27]:
model.get_nearest_neighbors('garlic')

[(0.9173319935798645, 'garlic.'),
 (0.8959659934043884, 'garli'),
 (0.8866293430328369, 'garlic,.'),
 (0.8809301257133484, 'garlic,&'),
 (0.8799318671226501, 'garlicand'),
 (0.8730747699737549, 'garlic,oregano'),
 (0.8729045391082764, 'garlic,'),
 (0.8690754175186157, 'garlice'),
 (0.8685024380683899, 'shallots,garlic'),
 (0.8647711873054504, 'shallots/garlic')]

## Eksport zanurzeń

In [9]:
with open("embeddings.txt", 'w') as f:
    for i in model.get_words():
        f.write(i)
        
        for j in model.get_word_vector(i):
            f.write(",")
            f.write(str(j))
        f.write("\n")

In [12]:
df["directions"][0]

'In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine. Stir over medium heat until mixture bubbles all over top. Boil and stir 5 minutes more. Take off heat. Stir in vanilla and cereal; mix well. Using 2 teaspoons, drop and shape into 30 clusters on wax paper. Let stand until firm, about 30 minutes.'

In [ ]:
import json
from tqdm import tqdm

corpus = []
json_fail_count = 0
json_fail = []
for ingredients_entities in tqdm(df["ingredients_entities"].tolist()):
    sentence = []
    try:
        for entity in json.loads(ingredients_entities):
            if entity["type"] == "FOOD":
                sentence.append("_".join(normalize_name(entity["entity"]).split(" ")))
        corpus.append(" ".join(sentence).lower())
    except:
        json_fail.append(ingredients_entities)
        json_fail_count += 1

In [19]:
with open('dataset_new_forfasttext_instructions.csv', 'w') as csvfile:
    for i in tqdm(df["directions"]):
        if type(i) !=  str:
            print("ERR",i)
            continue
        csvfile.write(i.lower())
        csvfile.write("\n")

 81%|████████  | 1809148/2231142 [00:03<00:01, 421758.64it/s]

ERR nan


100%|██████████| 2231142/2231142 [00:04<00:00, 509885.51it/s]


In [23]:
df.iloc[1809149,]

title                                                      Fancy Meatloaf
ingredients             1/2 loaf Italian bread, crust removed, torn in...
directions              Preheat the oven to 375 degrees. Soak the brea...
link                                  cooking.nytimes.com/recipes/1012686
source                                                          Recipes1M
NER                     ["Italian bread", "milk", "ground beef", "grou...
ingredients_entities    [{"start": 0, "end": 3, "type": "QUANTITY", "e...
Name: 1809149, dtype: object